<a href="https://colab.research.google.com/github/pttxxxhit/Demographic-Data-Analyzer/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  # Install a specific stable version of TensorFlow
  !pip install tensorflow==2.20.0
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# Import specific components from Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# 3.1) Convertir “ham”→0 y “spam”→1
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data ['label'] = test_data ['label'].map({'ham': 0, 'spam': 1})

# 3.2) Separar textos y etiquetas
train_texts  = train_data['message'].tolist()
train_labels = train_data['label'].values
test_texts   = test_data ['message'].tolist()
test_labels  = test_data ['label'].values

# 3.3) Transformar textos en números con Tokenizer
vocab_size = 10000
max_len    = 100

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(train_texts)

train_seq = tokenizer.texts_to_sequences(train_texts)
train_pad = pad_sequences(train_seq, maxlen=max_len, padding='post')

test_seq = tokenizer.texts_to_sequences(test_texts)
test_pad = pad_sequences(test_seq,  maxlen=max_len, padding='post')

# Ver formas finales
print("train_pad:", train_pad.shape, "train_labels:", train_labels.shape)
print(" test_pad:", test_pad.shape,  " test_labels:", test_labels.shape)

In [ ]:
# Load the datasets into pandas DataFrames
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data  = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Display the first few rows of the training data to verify
display(train_data.head())

In [ ]:
# 4.1) Crear la red neuronal
model = Sequential([
    Embedding(vocab_size, 16, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 4.2) Entrenar la red
history = model.fit(
    train_pad,
    train_labels,
    epochs=10,
    validation_data=(test_pad, test_labels),
    verbose=2
)

# 4.3) Función que predice un solo mensaje
def predict_message(text):
    seq  = tokenizer.texts_to_sequences([text])
    pad  = pad_sequences(seq, maxlen=max_len, padding='post')
    prob = float(model.predict(pad)[0][0])
    label = 'spam' if prob > 0.5 else 'ham'
    return [prob, label]

# Probar la función con ejemplos
print(predict_message("Congratulations! You've won a free ticket. Call now."))
print(predict_message("¿Nos vemos mañana en el cine?"))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):



  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
